# Fine tune Flux with DreamBooth LoRA Hugging Face Diffusers

### Download the dataset

In this example, we use a dataset of dog images. You can replace this with your own dataset. Only 5 images are needed for Dreambooth fine-tuning.

In [22]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'/home/ubuntu/temp/flux-fine-tune/dog'

In [23]:
# training job will fail if the dog/ contains a sub directory
!rm -rf dog/.cache

Import necessary libraries. We use dot env to load api keys from environment variables so they are not hardcoded in the notebook. We use the sagemaker modeltrainer api to kick off the training job.

In [24]:
import os
from dotenv import load_dotenv
from sagemaker.modules import Session
from sagemaker.modules.train import ModelTrainer
from sagemaker.modules.distributed import Torchrun
from sagemaker.modules.configs import (
    CheckpointConfig,
    Compute, 
    SourceCode, 
    InputData, 
    StoppingCondition, 
    S3DataSource
)

load_dotenv()  

True

In [25]:
environment = {
    'HF_TOKEN': os.environ["HF_TOKEN"],
    'WANDB_API_KEY': os.environ["WANDB_API_KEY"],
}

The StoppingCondition max_runtime_in_seconds allows us to use warm pools for the training job. This reduces startup time for the training job if we are going to be using the same instance type for multiple training jobs.

For this example script we use a single p4de instance and we run on a single GPU. You can also train on multple GPUs by editing the script.sh in the scripts folder and uncommenting the multi GPU launch command and commenting the single GPU launch command.

This script will also run on a p5 instance.

You can adjust the training parameters in the script.sh file. The training parameters are passed to the training script as command line arguments. The training script is located in the scripts folder.

In [26]:
sess = Session()
bucket = sess.default_bucket()
base_job_name = "flux-fine-tune"

pytorch_image = '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.6.0-gpu-py312-cu126-ubuntu22.04-sagemaker'
image = '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.7.1-gpu-py312-cu128-ubuntu22.04-sagemaker'
# define the script to be run
source_code = SourceCode(
    source_dir="scripts/",
    entry_script="script.sh",
)

compute = Compute(
    instance_count=1,
    instance_type="ml.p4de.24xlarge",
    keep_alive_period_in_seconds=3600,
)

stopping_condition = StoppingCondition(max_runtime_in_seconds=18000)

checkpoint_config = CheckpointConfig(
    s3_uri=f"s3://{sess.default_bucket()}/{base_job_name}/checkpoints/",
)

# define the ModelTrainer
model_trainer = ModelTrainer(
    sagemaker_session=sess,
    training_image=image,
    source_code=source_code,
    base_job_name=base_job_name,
    compute=compute,
    environment=environment,
    stopping_condition=stopping_condition,
    checkpoint_config=checkpoint_config,
)

[08/12/25 00:08:40] WARNING  Role not provided. Using default role:                            ]8;id=342163;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=769636;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#505\505]8;;\
                             arn:aws:iam::364430515305:role/macbook23                                              

                    WARNING  OutputDataConfig not provided. Using default:                     ]8;id=879054;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=417371;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#567\567]8;;\
                             s3_output_path='s3://sagemaker-us-east-1-364430515305/flux-fine-t                     
                             une' kms_key_id=None compression_type='GZIP'                                          

                    INFO     Training image URI:                                               ]8;id=484737;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=852401;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#588\588]8;;\
                             763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.7                     
                             .1-gpu-py312-cu128-ubuntu22.04-sagemaker                                              

Upload the dataset to S3. The S3 URI should not have a trailing slash.


In [27]:
from sagemaker.s3 import S3Uploader

S3Uploader.upload(
    local_path="./dog/",
    # cannot have a trailing slash in the S3 URI
    desired_s3_uri=f"s3://{bucket}/flux/dog",
)

's3://sagemaker-us-east-1-364430515305/flux/dog'

In [28]:
data = InputData(
    channel_name="train",
    data_source=S3DataSource(
        s3_data_type="S3Prefix",
        s3_uri=f"s3://{sess.default_bucket()}/flux/dog",
        s3_data_distribution_type="FullyReplicated",
    ) 
)

Kick off the training job. The training job will use the script.sh file in the scripts folder to run the training script. 

In [29]:
model_trainer.train(input_data_config=[data], wait=False)

[08/12/25 00:08:42] INFO     Creating training_job resource.                                     ]8;id=334261;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=831845;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker_core/main/resources.py#28491\28491]8;;\

[08/12/25 00:08:42] WARNING  Not displaing the training container logs as 'wait' is set to     ]8;id=409763;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=10930;file:///home/ubuntu/temp/flux-fine-tune/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#834\834]8;;\
                             False.                                                                                